# Most Visited Monopoly Properties

---

## Import packages

In [1]:
# JSON manipulation
import json

# Randomization
import random

# Matrices manipulation
import numpy as np

# Data frame manipulation
import pandas as pd

## Load the data

### The `chance` and `community chest`

In [3]:
# Open JSON file
card = open('../data/card/chance_community_card.json')
# Returns JSON object as a dictionary
data_card = json.load(card)

In [4]:
# Chance and community cards
data_card['chance_card']

[{'number': 1,
  'naration': 'Advance to Go (Collect Â£200)',
  'action': 'move_money',
  'asset': None,
  'money': {'amount': 200,
   'cash_in': True,
   'requisite': {'status': False, 'condition': None}},
  'move': {'properties': 1, 'forward': False, 'step': None}},
 {'number': 2,
  'naration': 'Advance to Trafalgar Square. If you pass Go, collect Â£200',
  'action': 'move_money',
  'asset': None,
  'money': {'amount': 200,
   'cash_in': True,
   'requisite': {'status': True, 'condition': 'pass_go'}},
  'move': {'properties': 25, 'forward': False, 'step': None}},
 {'number': 3,
  'naration': 'Advance to Mayfair',
  'action': 'move',
  'asset': None,
  'money': {'amount': 0,
   'cash': None,
   'requisite': {'status': False, 'condition': None}},
  'move': {'properties': 40, 'forward': False, 'step': None}},
 {'number': 4,
  'naration': 'Advance to Pall Mall. If you pass Go, collect Â£200',
  'action': 'move_money',
  'asset': None,
  'money': {'amount': 200,
   'cash_in': True,
   're

### The `spaces`

In [5]:
# Open JSON file
space = open('../data/asset/board_spaces.json')
# Returns JSON object as a dictionary
data_space = json.load(space)

In [6]:
# Spaces
data_space

[{'order': 1,
  'name': 'Go',
  'space_type': 'corner_square',
  'color': 'white',
  'money': {'amount': 200, 'cash_in': True}},
 {'order': 2,
  'name': 'Old Kent Road',
  'space_type': 'road',
  'color': 'brown',
  'money': {'amount': 60, 'cash_in': False},
  'rent': {'alone': 2,
   'monopoly': 4,
   'with_house': {'house_1': 10,
    'house_2': 30,
    'house_3': 90,
    'house_4': 160,
    'hotel': 250}},
  'prices': {'printed_price': 60, 'mortgage_value': 30, 'building_costs': 50}},
 {'order': 3,
  'name': 'Community Chest',
  'space_type': 'community_chest',
  'color': 'white',
  'money': {'amount': 0, 'cash_in': False}},
 {'order': 4,
  'name': 'Whitechapel Road',
  'space_type': 'road',
  'color': 'brown',
  'money': {'amount': 60, 'cash_in': False},
  'rent': {'alone': 4,
   'monopoly': 8,
   'with_house': {'house_1': 20,
    'house_2': 60,
    'house_3': 180,
    'house_4': 320,
    'hotel': 450}},
  'prices': {'printed_price': 60, 'mortgage_value': 30, 'building_costs': 50}},


## Take `chance` and `community chest` cards

### Functions for taking a card

In [8]:
# Update spaces
def update_space(
    last_move: int,
    dices: list,
    data_space: dict
    ):
    # Updated board based latest move
    l_board = (data_space[last_move - 1:] + data_space[:last_move - 1])
    # Current space
    d_current_space = [l_board[np.sum(dices) % len(l_board)]]
    
    return d_current_space

In [9]:
# Find the shortest spaces in circular list
def distance_space(
    length_space: int,
    first_index: int,
    second_index: int
    ):
    # Calculate the distance
    distance = abs(first_index - second_index)
    
    return min(length_space - distance, distance)

In [10]:
# Find the shortest spaces for spaces in circular list
def shortest_space(
    source: int,
    targets: list,
    data_space: dict
    ):
    # Create a dictionary
    d_targets = {key: None for key in targets}
    
    # Loop the targets
    for target in d_targets.keys():
        # Calculate the minimum distance
        shortest = distance_space(
            length_space = len(data_space),
            first_index = source,
            second_index = target
        )
        
        # Update the dictionary
        d_targets.update(
            {
                target: shortest
            }
        )
        
    # Get the best space
    space = min(d_targets, key = d_targets.get)
    
    # Forward-backward space
    forward_space = update_space(
        last_move = source,
        dices = d_targets[space],
        data_space = data_space
    )
    
    # Forward-backward status
    forward_status = False
    if forward_space[0]['order'] == space:
        forward_status = True
    
    return (space, forward_status)

In [12]:
# Select a card and show the result
def take_cards(
    subiteration_dict: dict,
    card: bool,
    chance_card: bool,
    data_card: dict,
    data_space: dict
    ):
    # Card type
    card = 'community_card'
    
    # Chance or community chest card
    if chance_card:
        # Update the card type
        card = 'chance_card'
        # Select the card
        chance_cards = data_card['chance_card']
        selected_card = random.choice(chance_cards)
    else:
        # Select the card
        community_cards = data_card['community_card']
        selected_card = random.choice(community_cards)
    
    # CORE ALGORITHM
    # Update the card status
    subiteration_dict['move'][card] = selected_card['number']
    # Move the space
    if selected_card['action'] in ['move', 'move_money']:
        # Last space
        last_space = subiteration_dict['move']['after']['real']
        if subiteration_dict['move']['after']['updated'] != None:
            last_space = subiteration_dict['move']['after']['updated']
        # Properties
        property_index = selected_card['move']['properties']

        # Move few step
        if selected_card['move']['properties'] == None:
            # Steps
            step = selected_card['move']['step']

            # Multiplier
            multiplier = -1
            # Update the multiplier
            if selected_card['move']['forward']:
                multiplier = 1

            # Current space
            d_current_space_card = update_space(
                last_move = last_space,
                dices = (multiplier * step),
                data_space = data_space
            )

            # Update the last space
            if card:
                subiteration_dict['move']['after']['updated'] = d_current_space_card[0]['order']
                subiteration_dict['move']['forward']['updated'] = selected_card['move']['forward']
            else:
                pass

        # Move to the specific space
        else:
            # Go to nearest property (998: utilities, 999: stations)
            if property_index in [998, 999]:
                # Get a list of spaces
                targets = {
                    998: [13, 29],
                    999: [6, 16, 26, 36]
                }
                # Get the shortest space
                update_index, forward_stat = shortest_space(
                    source = last_space,
                    targets = targets[property_index],
                    data_space = data_space
                )
                # Update the last space
                if card:
                    subiteration_dict['move']['after']['updated'] = update_index
                    subiteration_dict['move']['forward']['updated'] = forward_stat
                else:
                    pass

            # Go to specific property
            else:
                # Update the last space
                if card:
                    subiteration_dict['move']['after']['updated'] = property_index
                    subiteration_dict['move']['forward']['updated'] = True
                else:
                    pass
    
    return subiteration_dict

### Simulation

In [13]:
# Sample data
sample = {
    'subiteration': 1,
    'dices': [2, 3],
    'round': 1,
    'move': {
        'chance_card': None,
        'community_card': None,
        'jail': {
            'status': False,
            'number': 0
        },
        'before': 30,
        'after': {
            'real': 8,
            'updated': None
        },
        'forward': {
            'real': None,
            'updated': None
        }
    }
}

In [14]:
# Simulation
take_cards(
    subiteration_dict = sample,
    card = False,
    chance_card = True,
    data_card = data_card,
    data_space = data_space
)

{'subiteration': 1,
 'dices': [2, 3],
 'round': 1,
 'move': {'chance_card': 4,
  'community_card': None,
  'jail': {'status': False, 'number': 0},
  'before': 30,
  'after': {'real': 8, 'updated': 12},
  'forward': {'real': None, 'updated': True}}}

## Players movement

### Functions for movement

In [28]:
# Rounds on Monopoly board
def round_spaces(
    dict_spaces: dict
    ):
    # Latest round
    latest_round = dict_spaces['round']
    
    # Update status
    updated_status = False
    # After-before spaces
    before = dict_spaces['move']['before']
    after = dict_spaces['move']['after']['real']
    if dict_spaces['move']['after']['updated'] != None:
        after = dict_spaces['move']['after']['updated']
        updated_status = True
    
    # Create a list
    l = list(range(before, after))
    # Dice roll
    dices_roll = abs(before - after)
    
    # Jail status
    jail_status = dict_spaces['move']['jail']['status']
    # Chance card
    chance_card = dict_spaces['move']['chance_card']
    # Community chest card
    community_card = dict_spaces['move']['community_card']
    
    # If player(s) is on a 'Jail'
    if jail_status == False:
        if (chance_card != None) and (community_card != None):
            # Check condition
            if len(l) == 0:
                # Move forward
                if dices_roll > 12:
                    dict_spaces['round'] += 1
                # Stay
                else:
                    pass
            else:
                # Move backward
                if dices_roll > 12:
                    dict_spaces['round'] -= 1
                # Stay
                else:
                    pass
        else:
            if (chance_card != None):
                
    else:
        pass
    
    return dict_spaces

In [52]:
(12 != None) and (None != None)

False

In [30]:
# Player(s) movement
def move_spaces(
    dict_latest: dict,
    dices: int,
    subiteration: int,
    three_times: bool,
    card: bool,
    data_card: dict,
    data_space: dict
    ):
    # Template of players' iteration
    template_iter = {
        'subiteration': subiteration,
        'dices': dices,
        'round': 0,
        'move': {
            'chance_card': None,
            'community_card': None,
            'jail': {
                'status': False,
                'number': 0
            },
            'before': None,
            'after': {
                'real': None,
                'updated': None
            },
            'forward': {
                'real': None,
                'updated': None
            }
        }
    }
    
    # Jail status and number
    jail_status, jail_trial = dict_latest['move']['jail'].values()
    
    # latest space position
    last_move = dict_latest['move']['after']['real']
    if dict_latest['move']['after']['updated'] != None:
        last_move = dict_latest['move']['after']['updated']
    
    # If player(s) is on jail space
    if jail_status:
        if (len(np.unique(dices)) != 1):
            if (jail_trial < 2):
                # Update dictionary
                template_iter['move']['jail']['status'] = jail_status
                template_iter['move']['jail']['number'] = jail_trial + 1
                template_iter['move']['before'] = last_move
                template_iter['move']['after']['real'] = last_move
            else:
                # Current space
                d_current_space = update_space(
                    last_move = last_move,
                    dices = dices,
                    data_space = data_space
                )
                # Update dictionary
                template_iter['move']['before'] = last_move
                template_iter['move']['after']['real'] = d_current_space[0]['order']
                template_iter['move']['forward']['real'] = True
        else:
            # Current space
            d_current_space = update_space(
                last_move = last_move,
                dices = dices,
                data_space = data_space
            )
            # Update dictionary
            template_iter['move']['before'] = last_move
            template_iter['move']['after']['real'] = d_current_space[0]['order']
            template_iter['move']['forward']['real'] = True
    
    # If player(s) is not on jail space
    else:
        # If player(s) throw the same number three times
        if three_times:
            template_iter['move']['jail']['status'] = True
            template_iter['move']['after']['real'] = 11
            template_iter['move']['before'] = last_move
            template_iter['move']['forward']['real'] = False
        
        # Otherwise
        else:
            # List of space orders based on space type
            d_space = {
                'community_chest': [space['order'] for space in data_space if space['space_type'] == 'community_chest'],
                'chance': [space['order'] for space in data_space if space['space_type'] == 'chance'],
                'road': [space['order'] for space in data_space if space['space_type'] == 'road']
            }

            # Current space
            d_current_space = update_space(
                last_move = last_move,
                dices = dices,
                data_space = data_space
            )
            
            # Update dictionary
            template_iter['move']['before'] = last_move
            template_iter['move']['forward']['real'] = True

            # Player is on 'Go to Jail' space
            if d_current_space[0]['order'] == 31:
                # Update dictionary
                template_iter['move']['jail']['status'] = True
                template_iter['move']['after']['real'] = d_current_space[0]['order']
                template_iter['move']['after']['updated'] = 11
                template_iter['move']['forward']['real'] = True
                template_iter['move']['forward']['updated'] = False

            # Player is not on 'Go to Jail' space
            else:
                # Update dictionary
                template_iter['move']['after']['real'] = d_current_space[0]['order']
                template_iter['move']['forward']['real'] = True

                # Player is on 'Community Chest' space
                if d_current_space[0]['order'] in d_space['community_chest']:
                    # Update dictionary
                    template_iter = take_cards(
                        subiteration_dict = template_iter,
                        card = card,
                        chance_card = False,
                        data_card = data_card,
                        data_space = data_space
                    )
                if d_current_space[0]['order'] in d_space['chance']:
                    # Update dictionary
                    template_iter = take_cards(
                        subiteration_dict = template_iter,
                        card = card,
                        chance_card = True,
                        data_card = data_card,
                        data_space = data_space
                    )
                else:
                    pass
    
    # Latest round
    template_iter['round'] = dict_latest['round']
    
    # Update the round
    template_iter = round_spaces(dict_spaces = template_iter)
    
    return template_iter

### Simulation

In [41]:
# Sample data
sample = {
    'subiteration': 1,
    'dices': [2, 3],
    'round': 1,
    'move': {
        'chance_card': None,
        'community_card': None,
        'jail': {
            'status': False,
            'number': 0
        },
        'before': 30,
        'after': {
            'real': 35,
            'updated': None
        },
        'forward': {
            'real': None,
            'updated': None
        }
    }
}

In [42]:
# Simulation
move_spaces(
    dict_latest = sample,
    dices = [3, 3],
    subiteration = 6,
    three_times = False,
    card = True,
    data_card = data_card,
    data_space = data_space
)

{'subiteration': 6,
 'dices': [3, 3],
 'round': 2,
 'move': {'chance_card': None,
  'community_card': None,
  'jail': {'status': False, 'number': 0},
  'before': 35,
  'after': {'real': 1, 'updated': None},
  'forward': {'real': True, 'updated': None}}}

## Core simulation

In [43]:
# Monopoly simulation
def monopoly_simulation(
    num_player: int,
    iteration: int,
    card: bool
    ):
    # String of players
    player_str = ['Player ' + str(i) for i in range(1, num_player + 1)]
    
    # A dictionary for players
    obj = {
        'players': num_player,
        'iteration': iteration,
        'simulation': dict([(idx, dict([('iterations', [])])) for idx in player_str])
    }
    
    # Loop for iterations
    for turn in range(iteration):
        # Loop for players
        for player in player_str:
            # Length of iterations in each player
            length_iter = len(obj['simulation'][player]['iterations'])
            
            if turn == 0:
                # Initial move
                d_last_move = {
                    'subiteration': 0,
                    'dices': [0, 0],
                    'round': 0,
                    'move': {
                        'chance_card': None,
                        'community_card': None,
                        'jail': {
                            'status': False,
                            'number': 0
                        },
                        'before': 1,
                        'after': {
                            'real': 1,
                            'updated': None
                        }
                    }
                }
            else:
                # Check latest move
                d_last_move = obj['simulation'][player]['iterations'][-1]['data'][-1]
            
            # CORE ALGORITHM
            # Template for subiteration
            subiteration_temp = {'iteration': turn, 'data': [d_last_move]}
            # Roll the dices
            dices = random.choices(population = range(1, 7), k = 2)
            # Subiteration
            loop = 1

            # While loop
            while (len(np.unique(dices)) == 1) and (loop < 3):
                # Get the subiteration
                subiteration_data = move_spaces(
                    dict_latest = subiteration_temp['data'][-1],
                    dices = dices,
                    subiteration = loop,
                    three_times = False,
                    card = card,
                    data_card = data_card,
                    data_space = data_space
                )

                # Append the subiteration data
                subiteration_temp['data'].append(subiteration_data)

                # Update the indexer
                dices = random.choices(population = range(1, 7), k = 2)
                loop += 1
            
            else:
                # If same numbers are found three times
                if (len(np.unique(dices)) == 1) and (loop > 2):
                    three_times = True
                # Next move
                else:
                    three_times = False
                
                # Get the subiteration
                subiteration_data = move_spaces(
                    dict_latest = subiteration_temp['data'][-1],
                    dices = dices,
                    subiteration = loop,
                    three_times = three_times,
                    card = card,
                    data_card = data_card,
                    data_space = data_space
                )

                # Append the subiteration data
                subiteration_temp['data'].append(subiteration_data)
            
            # Remove first element in data of 'subiteration_temp'
            subiteration_temp['data'] = subiteration_temp['data'][1:]
            
            # Append the obj
            obj['simulation'][player]['iterations'].append(subiteration_temp)
    
    return obj

### Simulation

In [44]:
# Parameters
num_player = 5
iteration = 200

# Simulation
data_simulation = monopoly_simulation(
    num_player = num_player,
    iteration = iteration,
    card = True
)

In [45]:
# Show the data
data_simulation

{'players': 5,
 'iteration': 200,
 'simulation': {'Player 1': {'iterations': [{'iteration': 0,
     'data': [{'subiteration': 1,
       'dices': [1, 2],
       'round': 0,
       'move': {'chance_card': None,
        'community_card': None,
        'jail': {'status': False, 'number': 0},
        'before': 1,
        'after': {'real': 4, 'updated': None},
        'forward': {'real': True, 'updated': None}}}]},
    {'iteration': 1,
     'data': [{'subiteration': 1,
       'dices': [5, 2],
       'round': 0,
       'move': {'chance_card': None,
        'community_card': None,
        'jail': {'status': False, 'number': 0},
        'before': 4,
        'after': {'real': 11, 'updated': None},
        'forward': {'real': True, 'updated': None}}}]},
    {'iteration': 2,
     'data': [{'subiteration': 1,
       'dices': [1, 2],
       'round': 0,
       'move': {'chance_card': None,
        'community_card': None,
        'jail': {'status': False, 'number': 0},
        'before': 11,
        'a

### Convert JSON into data frame

In [46]:
# Flatten the JSON
def flatten_json(
    d: dict
    ):
    # Normalize JSON into data frame
    f = pd.json_normalize(d, sep = '_')
    # Data frame into JSON
    result_json = f.to_dict(orient = 'records')[0]
    
    return result_json

In [47]:
# Players
players = data_simulation['simulation'].keys()

# Object to store data
obj_simulation = []

# Loop the players
for player in players:
    # Number of iteration
    data_iterations = data_simulation['simulation'][player]['iterations']
    
    # Loop the iteration
    for iter_ in range(len(data_iterations)):
        # Selected iteration
        selected_iteration = data_iterations[iter_]
        # Iteration
        n_iteration = selected_iteration['iteration']
        
        # Loop the subiteration
        for subiter_ in range(len(selected_iteration['data'])):
            # Selected subiteration
            selected_subiteration = selected_iteration['data'][subiter_]
            # Flatten the JSON
            subiteration_reformat = flatten_json(selected_subiteration)
            # Add iteration and player
            new_data = {**{'player': player, 'iteration': n_iteration}, **subiteration_reformat}
            
            # Append into data object
            obj_simulation.append(new_data)

In [48]:
# Data frame
df = pd.DataFrame(
    data = obj_simulation
)

In [50]:
# Show the data
df.head(50)

,player,iteration,subiteration,dices,round,move_chance_card,move_community_card,move_jail_status,move_jail_number,move_before,move_after_real,move_after_updated,move_forward_real,move_forward_updated
0,Player 1,0,1,"[1, 2]",0,NaN,NaN,False,0,1,4,NaN,True,None
1,Player 1,1,1,"[5, 2]",0,NaN,NaN,False,0,4,11,NaN,True,None
2,Player 1,2,1,"[1, 2]",0,NaN,NaN,False,0,11,14,NaN,True,None
3,Player 1,3,1,"[4, 6]",0,NaN,NaN,False,0,14,24,NaN,True,None
4,Player 1,4,1,"[3, 6]",0,NaN,NaN,False,0,24,33,NaN,True,None
5,Player 1,5,1,"[2, 3]",0,NaN,NaN,False,0,33,38,NaN,True,None
6,Player 1,6,1,"[4, 4]",1,NaN,NaN,False,0,38,6,NaN,True,None
7,Player 1,6,2,"[3, 6]",1,NaN,NaN,False,0,6,15,NaN,True,None
8,Player 1,7,1,"[3, 4]",1,NaN,NaN,False,0,15,22,NaN,True,None
9,Player 1,8,1,"[2, 6]",1,NaN,NaN,False,0,22,30,NaN,True,None


In [171]:
dict(df['move_after_real'].value_counts())

{11: 8439,
 19: 3391,
 21: 3365,
 17: 3252,
 23: 3215,
 20: 3214,
 18: 3185,
 22: 3178,
 26: 3100,
 24: 3070,
 31: 3050,
 27: 3015,
 32: 3006,
 29: 2999,
 33: 2977,
 25: 2969,
 28: 2953,
 16: 2950,
 30: 2932,
 13: 2893,
 35: 2876,
 15: 2855,
 34: 2847,
 36: 2756,
 10: 2692,
 9: 2691,
 14: 2646,
 8: 2623,
 37: 2616,
 6: 2570,
 12: 2547,
 4: 2537,
 5: 2534,
 7: 2528,
 40: 2525,
 3: 2524,
 39: 2515,
 2: 2474,
 38: 2445,
 1: 2418}